In [1]:
import numpy as np
import helper
import pickle
import datetime
import os.path
import time
import pandas as pd
import sqlite3
import pandas as pd
import re

In [2]:
def extract_price_and_currency(df):
    # Use regex to extract currency symbol (assumes symbol is at the start)
    df['currency'] = df['price'].str.extract(r'^([^\d]+)')  # non-digit characters at start
    # Extract numeric part, handle empty strings after replacement
    price_numeric = df['price'].str.replace(r'[^\d.]', '', regex=True)
    # Replace empty strings with NaN explicitly
    price_numeric = price_numeric.replace('', np.nan)
    # Assign and convert safely
    df['price_float'] = price_numeric.astype(float)
    # Drop rows where conversion failed
    df = df.dropna(subset=['price_float'])
    return df

def query_flights_from_db(db_filename, mode, folder_location, origin=None, destination=None, max_price=None, after_date=None):
    # Connect to the database
    # print(f"{folder_location}{db_filename}_{mode}.db")
    conn = sqlite3.connect(f"{folder_location}{db_filename}_{mode}.db")
    cursor = conn.cursor()

    # Build query with optional conditions
    query = "SELECT * FROM data_table WHERE 1=1"
    params = []

    if origin:
        query += " AND origin = ?"
        params.append(origin)
    if destination:
        query += " AND destination = ?"
        params.append(destination)
    if max_price:
        query += " AND CAST(price AS FLOAT) <= ?"
        params.append(max_price)
    if after_date:
        query += " AND departure_date >= ?"
        params.append(after_date)

    # Execute the query and load into DataFrame
    df = pd.read_sql_query(query, conn, params=params)
    print(f"\nTotal rows fetched into DataFrame: {len(df)}")

    # Close connection
    conn.close()
    return df


def filter_df(
    df,
    max_price=None,
    min_price=None,
    start_date=None,
    end_date=None,
    exact_origin=None,
    exact_destination=None,
    max_stops=None
):
    filtered_df = df.copy()

    if max_price is not None:
        filtered_df = filtered_df[pd.to_numeric(filtered_df['price_float'], errors='coerce') <= max_price]
    if min_price is not None:
        filtered_df = filtered_df[pd.to_numeric(filtered_df['price_float'], errors='coerce') >= min_price]
    if start_date is not None:
        filtered_df = filtered_df[filtered_df['departure_date'] >= start_date]
    if end_date is not None:
        filtered_df = filtered_df[filtered_df['departure_date'] <= end_date]
    if exact_origin:
        filtered_df = filtered_df[filtered_df['origin'] == exact_origin]
    if exact_destination:
        filtered_df = filtered_df[filtered_df['destination'] == exact_destination]
    if max_stops is not None:
        filtered_df = filtered_df[pd.to_numeric(filtered_df['stops'], errors='coerce') <= max_stops]

    print(f"Filtered rows remaining: {len(filtered_df)}")
    return filtered_df

In [3]:

results = query_flights_from_db(
    db_filename="dataDB",
    mode="intl",
    folder_location="./logs/",
    origin="ATL",
    destination="CUN",
    # max_price=450,
    # after_date="2025-07-01"
)
results = results[~results['price'].str.strip().isin(['', '$'])]
print(f"\nTotal rows after removing invalid price: {len(results)}")
results = extract_price_and_currency(results)
results = results.dropna(subset=['price_float'])
print(f"\nTotal rows after removing failed price conversion: {len(results)}")



Total rows fetched into DataFrame: 1715339

Total rows after removing invalid price: 1715330

Total rows after removing failed price conversion: 1715330


In [6]:
# results = extract_price_and_currency(results)
print(f"\nTotal rows now: {len(results)}")
# print(results.head())
results_nonstop = filter_df(results, max_price=300, max_stops=0)
results_nonstop = results_nonstop[results_nonstop["name"]!="Frontier"]
# results_nonstop = filter_df(results_nonstop, start_date="2025-03-01", end_date="2025-03-25",max_stops=0)
results_nonstop = results_nonstop.sort_values(by="price_float", ascending=True)
display(results_nonstop.head(50))


Total rows now: 1715330
Filtered rows remaining: 2324


,id,origin,destination,name,days,price,today,days_ahead,flight_duration,flight_depart,flight_arrive,stops,stops_info,departure_date,currency,price_float
93832,991776,ATL,CUN,"Delta, Aeromexico",4,$284,2024-06-10,69,2 hr 34 min,"8:26 AM on Mon, Aug 19","10:00 AM on Mon, Aug 19",0,,2024-08-19,$,284.0
93833,991777,ATL,CUN,"Delta, Aeromexico",4,$284,2024-06-10,69,2 hr 34 min,"9:45 AM on Mon, Aug 19","11:19 AM on Mon, Aug 19",0,,2024-08-19,$,284.0
93834,991778,ATL,CUN,"Delta, Aeromexico",4,$284,2024-06-10,69,2 hr 32 min,"11:28 AM on Mon, Aug 19","1:00 PM on Mon, Aug 19",0,,2024-08-19,$,284.0
94176,992120,ATL,CUN,"Delta, Aeromexico",4,$297,2024-06-10,98,2 hr 34 min,"9:45 AM on Tue, Sep 17","11:19 AM on Tue, Sep 17",0,,2024-09-17,$,297.0
94177,992121,ATL,CUN,"Delta, Aeromexico",4,$297,2024-06-10,98,2 hr 34 min,"11:21 AM on Tue, Sep 17","12:55 PM on Tue, Sep 17",0,,2024-09-17,$,297.0
94178,992122,ATL,CUN,"Delta, Aeromexico",4,$297,2024-06-10,98,2 hr 30 min,"1:21 PM on Tue, Sep 17","2:51 PM on Tue, Sep 17",0,,2024-09-17,$,297.0
94745,992689,ATL,CUN,"Delta, Aeromexico",4,$287,2024-06-10,110,2 hr 34 min,"9:45 AM on Sun, Sep 29","11:19 AM on Sun, Sep 29",0,,2024-09-29,$,287.0
94746,992690,ATL,CUN,"Delta, Aeromexico",4,$287,2024-06-10,110,2 hr 30 min,"1:21 PM on Sun, Sep 29","2:51 PM on Sun, Sep 29",0,,2024-09-29,$,287.0
94747,992691,ATL,CUN,"Delta, Aeromexico",4,$287,2024-06-10,110,2 hr 30 min,"6:25 PM on Sun, Sep 29","7:55 PM on Sun, Sep 29",0,,2024-09-29,$,287.0
95308,993252,ATL,CUN,"Delta, Aeromexico",4,$284,2024-06-10,83,2 hr 34 min,"8:26 AM on Mon, Sep 2","10:00 AM on Mon, Sep 2",0,,2024-09-02,$,284.0
